In [221]:
import numpy as np
import pandas as pd
import requests
import itertools

# Wybór najlepszego filmu

## API filmowe

Wykorzystamy IMDb API do pobrania informacji o filmach

In [2]:
api_key = '63a56440'
url = f'http://www.omdbapi.com/?apikey={api_key}'
params = {"t":"", "plot": "short", "r":"json"}

In [3]:
def fetch_movies(movies):
    movies_data = {}
    for movie in movies:
        params["t"] = movie
        response = requests.get(url, params=params).json()
        movies_data[movie] = response
    return movies_data

## Zapytanie użytkownika o filmy

Poprosimy użytkownika aby podał interesujące go filmy a następnie aby podał kryteria według których chce oceniać filmy

In [12]:
value = None
movies = []

print("Please enter a movie title to compare. When you finish press enter.\n")

while value != '':
    value = input()
    if value != '':
        movies.append(value)

movies_data = fetch_movies(movies)

Please enter a movie title to compare. When you finish press enter.

Harry Potter
Avengers
Duna



In [13]:
criteria = list(movies_data[movies[0]].keys())

for index, criterium in enumerate(criteria[1:]):
    print(f'{index + 1}. {criterium}')

value = None
chosen_criteria = []

print("\nPlease enter a criteria number to compare. When you finish press enter.\n")

while value != '':
    value = input()
    if value != '':
        chosen_criteria.append(criteria[int(value)])

1. Year
2. Rated
3. Released
4. Runtime
5. Genre
6. Director
7. Writer
8. Actors
9. Plot
10. Language
11. Country
12. Awards
13. Poster
14. Ratings
15. Metascore
16. imdbRating
17. imdbVotes
18. imdbID
19. Type
20. DVD
21. BoxOffice
22. Production
23. Website
24. Response

Please enter a criteria number to compare. When you finish press enter.

8
12
16



## Tworzenie par porównań

Następnym krokiem będzie utworzenie par do porównania dla użytkownika

In [14]:
movies_pairs = list(itertools.combinations(movies, 2))
criteria_pairs = list(itertools.combinations(chosen_criteria, 2))

print("Movies pairs: ", movies_pairs)
print("\nCriteria pairs: ", criteria_pairs)

Movies pairs:  [('Harry Potter', 'Avengers'), ('Harry Potter', 'Duna'), ('Avengers', 'Duna')]

Criteria pairs:  [('Actors', 'Awards'), ('Actors', 'imdbRating'), ('Awards', 'imdbRating')]


## Zebranie porównań od użytkownika

Poprosimy użytkownika aby ocenił każdą parę filmów wg. wybranych przez niego kryteriów. We wsparciu wyboru wyświelimy użykownikowi informacje dotyczące danego filmu.

In [194]:
movies_matrices = [pd.DataFrame(columns=movies, index=movies) for _ in range(len(chosen_criteria))]
criteria_matrices = pd.DataFrame(columns=chosen_criteria, index=chosen_criteria)
print(chosen_criteria)

for i, criterium in enumerate(chosen_criteria):
    matrix = movies_matrices[i]
    np.fill_diagonal(matrix.values, 1)
    for pair in movies_pairs:
        val =  float(input(f'\nCriterium: {criterium}\n    {pair[0]}: {movies_data[pair[0]][criterium]}\n    {pair[1]}: {movies_data[pair[1]][criterium]}\n Compare first to second:\n'))
        matrix[pair[0]][pair[1]] = val
        matrix[pair[1]][pair[0]] = 1/val


['Actors', 'Awards', 'imdbRating']

Criterium: Actors
    Harry Potter: Daniel Radcliffe, Emma Watson, Rupert Grint
    Avengers: Linda Ballantyne, Tony Daniels, Ron Rubin
 Compare first to second:
2

Criterium: Actors
    Harry Potter: Daniel Radcliffe, Emma Watson, Rupert Grint
    Duna: Talis Talivaldis Abolins, Maya Arwen, Kaspars Dumburs
 Compare first to second:
2

Criterium: Actors
    Avengers: Linda Ballantyne, Tony Daniels, Ron Rubin
    Duna: Talis Talivaldis Abolins, Maya Arwen, Kaspars Dumburs
 Compare first to second:
0.5

Criterium: Awards
    Harry Potter: Nominated for 3 Oscars. 46 wins & 94 nominations total
    Avengers: N/A
 Compare first to second:
9

Criterium: Awards
    Harry Potter: Nominated for 3 Oscars. 46 wins & 94 nominations total
    Duna: N/A
 Compare first to second:
5

Criterium: Awards
    Avengers: N/A
    Duna: N/A
 Compare first to second:
0.5

Criterium: imdbRating
    Harry Potter: 8.1
    Avengers: 5.5
 Compare first to second:
6

Criterium: im

## Porównywanie parami kryteriów

In [209]:
np.fill_diagonal(criteria_matrices.values, 1)
for pair in criteria_pairs:
    val =  float(input(f'Rate importance: {pair[0]} to {pair[1]}:\n'))
    criteria_matrices[pair[0]][pair[1]] = val
    criteria_matrices[pair[1]][pair[0]] = 1/val

Rate importance: Actors to Awards:
0.5
Rate importance: Actors to imdbRating:
0.4
Rate importance: Awards to imdbRating:
0.5


# EVM ranking method

W tej części będziemy wizualizować kroki i wyniki procesu decyzyjnego <br>
Zaczniemy od zdefiniowania metody EVM i utworzenia macierzy z danymi

In [210]:
def count_evm(preferences: np.ndarray) -> np.ndarray:
    preferences = preferences.T
    n = preferences.shape[-1]
    geometric_average = np.prod(preferences, axis=-1)**(1/n)
    return geometric_average / np.sum(geometric_average)

In [211]:
priorities = pd.DataFrame(columns=movies, index=chosen_criteria)
criteria_importance = pd.DataFrame(columns=chosen_criteria)

In [212]:
criteria_matrices

,Actors,Awards,imdbRating
Actors,1,2,2.5
Awards,0.5,1,2
imdbRating,0.4,0.5,1


Wykorzystamy metodę EVM do policzenia wag dla każdego filmu dla wybranych przez użytkownika kategorii

In [213]:
criteria_priorities = pd.DataFrame(columns=chosen_criteria)
criteria_priorities.loc[0] = count_evm(criteria_matrices.to_numpy())
criteria_priorities = criteria_priorities.rename(index={0: "priority"})

comparison_priorities = pd.DataFrame(columns=chosen_criteria, index=movies)
for i, criterium in enumerate(chosen_criteria):
    matrix = movies_matrices[i]
    comparison_priorities[criterium] = count_evm(matrix.to_numpy())

In [214]:
comparison_priorities

,Actors,Awards,imdbRating
Harry Potter,0.493386,0.760789,0.755659
Avengers,0.1958,0.0816148,0.188134
Duna,0.310814,0.157596,0.056207


In [215]:
criteria_priorities

,Actors,Awards,imdbRating
priority,0.177494,0.30351,0.518996


Dalej przedstawimy znormalizowane wyniki, możemy teraz łatwo porównywać wartości z różnych wierszy tabeli.

In [216]:
for criterium in chosen_criteria:  
    comparison_priorities[criterium] = comparison_priorities[criterium].values * criteria_priorities[criterium].values
comparison_priorities

,Actors,Awards,imdbRating
Harry Potter,0.087573,0.230907,0.392184
Avengers,0.0347534,0.024771,0.0976408
Duna,0.0551676,0.0478322,0.0291712


### Wyniki
Ostatecznie pokażemy ranking filmów powstały na podstawie decyzji użytkownika

In [217]:
comparison_priorities["Goal"] = comparison_priorities.sum(axis=1)
comparison_priorities

,Actors,Awards,imdbRating,Goal
Harry Potter,0.087573,0.230907,0.392184,0.710664
Avengers,0.0347534,0.024771,0.0976408,0.157165
Duna,0.0551676,0.0478322,0.0291712,0.132171


### Walidacja wyników

W celu walidacji możemy obliczyć Consistency Ratio (CR) dla każdej macierzy porównań, wartość mniejszą od 0.1 możemy uznać za satysfakcjnonującą


In [218]:
RI = {1: 10**-8, 2: 10**-8, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.21, 7: 1.32, 8: 1.41, 9: 1.46, 10: 1.49}

def calc_consistency_ratio(preferences: np.ndarray, evm_ranking: np.ndarray) -> float:
    preferences = preferences.T
    n = preferences.shape[-1]
    cv = np.matmul(preferences, evm_ranking.T)
    cv_lambda = np.sum(cv)
    ci = (cv_lambda - n) / (n-1)
    return ci / RI[n]

In [219]:
print(calc_consistency_ratio(criteria_matrices.to_numpy(), criteria_priorities.to_numpy()))

0.021202645365081926
